# Get contributors for all GitHub organisation repositories

This Jupyter notebook illustrates how to use [PyGithub][pygithub], and the [GitHub REST APIv3][github-api] to get contributors for all GitHub organisation repositories.

[github-api]: https://docs.github.com/en/free-pro-team@latest/rest
[pygithub]: https://pygithub.readthedocs.io/

In [ ]:
from github import Github
from src import (
    extract_attribute_from_dict_of_paginated_lists,
    find_organisation_repos,
    get_items_for_all_repos,
    parse_api_token
)
import os

## Getting started

We first need to load environment variables, instantiate a `github.Github` class, and get all repositories within a Github organisation. Further details can be found in the ["Get GitHub organisation repositories" example Jupyter notebook][example-organisation_repositories].

[example-organisation_repositories]: ./example_get_organisation_repositories.ipynb

In [ ]:
# Load required environment variables
GITHUB_ORGANISATION = os.getenv("GITHUB_ORGANISATION")

# Instantiate a `github.Github` object to gain access to GitHub REST APIv3
g = Github(*parse_api_token(os.getenv("GITHUB_API_TOKEN")), per_page=100)

# Get all the repositories for `GITHUB_ORGANISATION`
organisation_repositories = find_organisation_repos(g, GITHUB_ORGANISATION)

## Getting contributors for each GitHub organisation repository

To obtain contributors for each GitHub organisation repository, use the wrapper function `src.get_items_for_all_repos`. This function accepts a `github.Github` object, a valid method, and a list or `github.PaginatedList.PaginatedList` object of PyGithub classes. The valid method must be a method within these PyGithub classes. It will return a dictionary, where the keys are the repository full names, and the values are `github.PaginatedList.PaginatedList` objects of PyGithub classes.

In our example, `organisation_repositories` is a `github.PaginatedList.PaginatedList` object of `github.Repository.Repository` objects, so our [valid method must be one from the `github.Repository.Repository` class][pygithub-repository]. As we want to get all contributors, the required method is `get_contributors`. This will return a dictionary where the each value is a `github.PaginatedList.PaginatedList` object of `github.NamedUser.NamedUser` objects.

[pygithub-repository]: https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html

In [ ]:
# Get all the contributors for these repositories
organisation_contributors = get_items_for_all_repos(g, "get_contributors", organisation_repositories)

### Extracting contributor information

As discussed, each value of `organisation_contributors` contains information about contributors in a specific repository as `github.NamedUser.NamedUser` objects.

To access this information, you can use the `extract_attribute_from_dict_of_paginated_lists` wrapper function. It accepts a dictionary where each value is a given `github.PaginatedList.PaginatedList` object containing any PyGithub class, and a named attribute in this PyGithub class. It returns a dictionary with the same keys, where the values are the attributes of this PyGithub class.

In our case `organisation_contributors` is a `github.PaginatedList.PaginatedList` object of `github.NamedUser.NamedUser` objects, so any [named attribute][pygithub-nameduser] for `github.NamedUser.NamedUser` can be used.

Let us extract the `name` attribute of all the GitHub organisation contributors, and show the results. `None` values usually mean you can't access that information for some reason, e.g. permissions, or that it doesn't exist.

[pygithub-nameduser]: https://pygithub.readthedocs.io/en/latest/github_objects/NamedUser.html

In [ ]:
# Get the names of all contributors for each repository in a GitHub organisation
organisation_contributors_name = extract_attribute_from_dict_of_paginated_lists(organisation_contributors, "name")
organisation_contributors_name